In [1]:
import pandas as pd
import re
import numpy as np
import pymorphy3
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
f = open("reviews_positive_test.txt", "r")
text = f.read()

In [3]:
text = text.split('<p>')
text.pop(0)

''

In [4]:
text

['Здравствуйте. Нужно было найти тарифы по карте в приложении, но не смогла сама. Обратилась за помощью в чат, где Виктория сообщила мне Подробнее с тарифом карты Вы можете ознакомиться в приложении по пути: Счёт -&gt; Карта -&gt; Тарифный план.</p>\n',
 'Обратилась в Альфа-банк в личных сообщениях в официальной группе в соцсетях 12 июля  2023 года в 17.03.</p>\n',
 'У меня ИП и это было проблемой, когда я в последний раз пытался оформить ипотеку. Сейчас стали появляться предложения даже для самозанятых, решил уточнить в Альфе, светит ли мне ипотека. Написал в чат поддержки 04.07 в районе полудня. Мне ответила операционист Анжелика. Она задала пару утоняющих вопросов, про срок жизни ИП, про налогообложение и внезапно про наличие детей. А потом расписала условия. Ставка, срок, максимальная сумма. Плюс комплект документов и требования к самому жилью. Ушел думать, банки явно стали лояльнее к предпринимателям.</p>\n',
 'В связи со сложным финансовым положением, не мог вовремя внести оплату

In [5]:
special_characters = ['</p>', '\xa0', '<br/>', '\n', '-&gt;']

In [6]:
for i in range(len(text)):


    for sp_ch in special_characters:
        text[i] = text[i].replace(sp_ch,' ')

    text[i] = text[i].lower()
    text[i] = re.sub(r'[^\w\s]', ' ', text[i])
    text[i] = re.sub(r'[^\w\s]+|[\d]+', r' ',text[i]).strip()
    text[i] = re.sub('[a-zA-Z\s]+', ' ', text[i])
    text[i] = re.sub(" +", " ", text[i])


In [14]:
text

['здравствуйте нужно было найти тарифы по карте в приложении но не смогла сама обратилась за помощью в чат где виктория сообщила мне подробнее с тарифом карты вы можете ознакомиться в приложении по пути счёт карта тарифный план',
 'обратилась в альфа банк в личных сообщениях в официальной группе в соцсетях июля года в',
 'у меня ип и это было проблемой когда я в последний раз пытался оформить ипотеку сейчас стали появляться предложения даже для самозанятых решил уточнить в альфе светит ли мне ипотека написал в чат поддержки в районе полудня мне ответила операционист анжелика она задала пару утоняющих вопросов про срок жизни ип про налогообложение и внезапно про наличие детей а потом расписала условия ставка срок максимальная сумма плюс комплект документов и требования к самому жилью ушел думать банки явно стали лояльнее к предпринимателям',
 'в связи со сложным финансовым положением не мог вовремя внести оплату по договору номер сотрудник с ов эльдар э ич связался со мной и совместно с

In [7]:
morph = pymorphy3.MorphAnalyzer()

def lemmatize(sentence):
    words = sentence.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [8]:
lemmatize_text = list(map(lemmatize,text))

In [9]:
print(lemmatize_text)

[['здравствуйте', 'нужно', 'быть', 'найти', 'тариф', 'по', 'карта', 'в', 'приложение', 'но', 'не', 'смочь', 'сам', 'обратиться', 'за', 'помощь', 'в', 'чат', 'где', 'виктория', 'сообщить', 'я', 'подробный', 'с', 'тариф', 'карта', 'вы', 'мочь', 'ознакомиться', 'в', 'приложение', 'по', 'путь', 'счёт', 'карта', 'тарифный', 'план'], ['обратиться', 'в', 'альфа', 'банк', 'в', 'личный', 'сообщение', 'в', 'официальный', 'группа', 'в', 'соцсеть', 'июль', 'год', 'в'], ['у', 'я', 'ип', 'и', 'это', 'быть', 'проблема', 'когда', 'я', 'в', 'последний', 'раз', 'пытаться', 'оформить', 'ипотека', 'сейчас', 'стать', 'появляться', 'предложение', 'даже', 'для', 'самозанятой', 'решить', 'уточнить', 'в', 'альф', 'светить', 'ли', 'я', 'ипотека', 'написать', 'в', 'чат', 'поддержка', 'в', 'район', 'полдень', 'я', 'ответить', 'операционист', 'анжелика', 'она', 'задать', 'пара', 'утонять', 'вопрос', 'про', 'срок', 'жизнь', 'ип', 'про', 'налогообложение', 'и', 'внезапно', 'про', 'наличие', 'ребёнок', 'а', 'потом', 

In [10]:
stop_words = ['в', 'на', 'с', 'и', 'я', 'о', 'к', 'а', 'за', 'по',
              'об', 'у', 'бы', 'от', 'ли', 'ул', 'что', 'со', 'из',
              'для', 'про', 'г', 'до', 'то', 'быть', 'по', 'мочь',
              'январь','февраль','март','апрель','май','июнь','июль',
              'август','сентябрь','октябрь','ноябрь','декабрь','год',
              'стать','это','когда','даже']


In [11]:
def delete_stop_words(sentence):
    res = list()
    for word in sentence:
        if word not in stop_words:
            res.append(word)
    return res

In [12]:
clean_text = list(map(delete_stop_words,lemmatize_text))

In [13]:
print(clean_text)

[['здравствуйте', 'нужно', 'найти', 'тариф', 'карта', 'приложение', 'но', 'не', 'смочь', 'сам', 'обратиться', 'помощь', 'чат', 'где', 'виктория', 'сообщить', 'подробный', 'тариф', 'карта', 'вы', 'ознакомиться', 'приложение', 'путь', 'счёт', 'карта', 'тарифный', 'план'], ['обратиться', 'альфа', 'банк', 'личный', 'сообщение', 'официальный', 'группа', 'соцсеть'], ['ип', 'проблема', 'последний', 'раз', 'пытаться', 'оформить', 'ипотека', 'сейчас', 'появляться', 'предложение', 'самозанятой', 'решить', 'уточнить', 'альф', 'светить', 'ипотека', 'написать', 'чат', 'поддержка', 'район', 'полдень', 'ответить', 'операционист', 'анжелика', 'она', 'задать', 'пара', 'утонять', 'вопрос', 'срок', 'жизнь', 'ип', 'налогообложение', 'внезапно', 'наличие', 'ребёнок', 'потом', 'расписать', 'условие', 'ставка', 'срок', 'максимальный', 'сумма', 'плюс', 'комплект', 'документ', 'требование', 'сам', 'жильё', 'уйти', 'думать', 'банк', 'явно', 'лояльный', 'предприниматель'], ['связь', 'сложный', 'финансовый', 'пол